In [1]:
import copy
Dataset=[({'面包'},{'牛奶'},{'茶'}), 
         ({'面包'},{'尿布'},{'啤酒'},{'茶'}),
         ({'牛奶'},{'尿布'},{'啤酒'}),
         ({'面包'},{'牛奶'},{'尿布'},{'茶'}),
         ({'面包'},{'尿布'},{'牛奶'}),
         ({'面包'},{'牛奶'},{'啤酒'},{'尿布'},{'茶'}),
         ({'啤酒'},{'牛奶'},{'茶'}),
         ({'面包'},{'茶'}),
         ({'面包'},{'尿布'},{'牛奶'},{'啤酒'},{'茶'}),
         ({'面包'},{'牛奶'})]

In [15]:
#统计支持度个数
candidate_dict={} #商品字典：5种商品
for set_origin in Dataset: #统计每种商品的支持度
    for thing in set_origin:
        if tuple(thing) not in candidate_dict.keys():
            candidate_dict[tuple(thing)]=0
        candidate_dict[tuple(thing)]+=1
prim_goods_dict=candidate_dict #置信度的分母
#print(prim_goods_dict)

all_frequent_dict={} #以字典的形式存放所有的频繁项集
while(len(candidate_dict)>1): #迭代到只剩一个候选项集
    #剪枝
    frequent_list=[]     #频繁项集
    non_frequent_list=[] #非频项集
    for item in candidate_dict.items():#每一个item是一个元组(('面包')，8)
        if (item[1]>0.35*len(Dataset)):
            frequent_list.append(set(item[0])) #大于支持度的项集放入频繁列表
            if(len(item[0])>1):#不要1项集
                all_frequent_dict[item[0]]=item[1]
        else: 
            non_frequent_list.append(set(item[0]))#把达不到minsup的项集放到非频繁项集里
    print("frequent_list:")
    #print(frequent_list)
    #print("non_frequent_list:")
    #print(non_frequent_list)
    #连接
    new_compose_list=[] #连接商品组合
    copy_frequent_list=copy.copy(frequent_list) #拷贝一份以便删除
    for element in frequent_list: #频繁项集里的每一项集与其他的项集组合，当他们两个补集等于2时可以连接
        copy_frequent_list.remove(element)#和剩下的搭配
        for goods in copy_frequent_list:
            if len(goods^element)==2:
                union_set=element|goods
                new_compose_list.append(union_set)
    #print("new_compose_list:")
    #print(new_compose_list)
    
    #再检查一次，看候选项集里有没有包含已经被刷掉的非频繁项集
    for compose_set in new_compose_list:
        for non_frequent_set in non_frequent_list:
            if non_frequent_set.issubset(compose_set):
                new_compose_list.remove(compose_set)
    #print("new_compose_list:")
    #print(new_compose_list)
    #清除重复项集(比如{'牛奶","面包","茶"}出现了2次)
    clean_compose_list=[]
    for compose in new_compose_list:
        if compose not in clean_compose_list:
            clean_compose_list.append(compose)
    #print(clean_compose_list)
    
    #统计支持度个数
    new_candidate_dict={}
    for new_combine in clean_compose_list:#new_combine是集合
        for original_set in [[list(Dataset[y][x])[0] for x in range(0,len(Dataset[y]))]for y in range(0,len(Dataset))]:#小集合变大集合
            if set(new_combine).issubset(original_set): #判断new_combine是不是子集，要转化为集合类型并用issubset方法判断
                if tuple(new_combine) not in new_candidate_dict.keys():
                    new_candidate_dict[tuple(new_combine)]=0
                new_candidate_dict[tuple(new_combine)]+=1
    #print("new_candidate_dict:")
    #print(new_candidate_dict)
    #print("\n")
    candidate_dict=new_candidate_dict #商品字典更新

print(all_frequent_dict,)


frequent_list:
frequent_list:
frequent_list:
{('牛奶', '面包'): 6, ('面包', '茶'): 6, ('面包', '尿布'): 5, ('牛奶', '茶'): 5, ('牛奶', '尿布'): 5, ('牛奶', '啤酒'): 4, ('尿布', '茶'): 4, ('啤酒', '茶'): 4, ('啤酒', '尿布'): 4, ('牛奶', '面包', '茶'): 4, ('牛奶', '面包', '尿布'): 4, ('尿布', '面包', '茶'): 4}


In [17]:
for key in all_frequent_dict.keys():
    print(key)
    if(len(key)==2):
        for good in key:
            confidence=all_frequent_dict[key]/prim_goods_dict[tuple({good})]
            print('置信度:',good,'|',key[key.index(good)-1],'=',confidence)
            lift=confidence/(prim_goods_dict[tuple({key[key.index(good)-1]})]/10)
            #print(good,'=',prim_goods_dict[tuple({key[key.index(good)-1]})])
            print('提升度:',good,'|',key[key.index(good)-1],'=',lift)
            #print('\n')
    if(len(key)==3):
        for good in key:
            copyKey=list(copy.copy(key))            
            copyKey.remove(good)
            print('先买',copyKey)
            if tuple(copyKey)in all_frequent_dict:
                confidence=all_frequent_dict[key]/all_frequent_dict[tuple(copyKey)]
            print('置信度:',key,'|',copyKey,'=',confidence)
            lift=confidence/(prim_goods_dict[tuple({good})]/10)
            #print(good,'=',prim_goods_dict[tuple({key[key.index(good)-1]})])
            print('提升度:',lift)

('牛奶', '面包')
置信度: 牛奶 | 面包 = 0.75
提升度: 牛奶 | 面包 = 0.9375
置信度: 面包 | 牛奶 = 0.75
提升度: 面包 | 牛奶 = 0.9375
('面包', '茶')
置信度: 面包 | 茶 = 0.75
提升度: 面包 | 茶 = 1.0714285714285714
置信度: 茶 | 面包 = 0.8571428571428571
提升度: 茶 | 面包 = 1.0714285714285714
('面包', '尿布')
置信度: 面包 | 尿布 = 0.625
提升度: 面包 | 尿布 = 1.0416666666666667
置信度: 尿布 | 面包 = 0.8333333333333334
提升度: 尿布 | 面包 = 1.0416666666666667
('牛奶', '茶')
置信度: 牛奶 | 茶 = 0.625
提升度: 牛奶 | 茶 = 0.8928571428571429
置信度: 茶 | 牛奶 = 0.7142857142857143
提升度: 茶 | 牛奶 = 0.8928571428571428
('牛奶', '尿布')
置信度: 牛奶 | 尿布 = 0.625
提升度: 牛奶 | 尿布 = 1.0416666666666667
置信度: 尿布 | 牛奶 = 0.8333333333333334
提升度: 尿布 | 牛奶 = 1.0416666666666667
('牛奶', '啤酒')
置信度: 牛奶 | 啤酒 = 0.5
提升度: 牛奶 | 啤酒 = 1.0
置信度: 啤酒 | 牛奶 = 0.8
提升度: 啤酒 | 牛奶 = 1.0
('尿布', '茶')
置信度: 尿布 | 茶 = 0.6666666666666666
提升度: 尿布 | 茶 = 0.9523809523809524
置信度: 茶 | 尿布 = 0.5714285714285714
提升度: 茶 | 尿布 = 0.9523809523809523
('啤酒', '茶')
置信度: 啤酒 | 茶 = 0.8
提升度: 啤酒 | 茶 = 1.142857142857143
置信度: 茶 | 啤酒 = 0.5714285714285714
提升度: 茶 | 啤酒 = 1.1428571428571428
('啤酒', '尿

In [ ]:
b=[list(Dataset[0][x])[0] for x in range(0,3)]
#print([[list(Dataset[y][x])[0] for x in range(0,len(Dataset[y]))] for y in range(0,len(Dataset))])
for original_set in [[list(Dataset[y][x])[0] for x in range(0,len(Dataset[y]))] for y in range(0,len(Dataset))]:
    print(set(original_set))

In [ ]:

print(Dataset[0])
data_list=[]
data_list.append(list(Dataset[0][x])[0] for x in range(0,len(Dataset[0])))
print(data_list)

In [ ]:
b=[list(Dataset[0][x]) for x in range(0,len(Dataset[0]))]
print(b)
print(list(Dataset[0][1])[0])